# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [33]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [34]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,8.57,83,80,8.92,NZ,1713734919
1,1,blackmans bay,-43.0167,147.3167,10.07,92,8,2.07,AU,1713734942
2,2,san patricio,28.0170,-97.5169,23.40,53,74,7.04,US,1713735513
3,3,edinburgh of the seven seas,-37.0676,-12.3116,16.93,79,44,7.96,SH,1713735224
4,4,hasaki,35.7333,140.8333,14.22,78,100,2.92,JP,1713735515


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True,
    frame_width = 800,
    frame_height = 600,
    scale = 0.8,
    color = "City",
    size = "Humidity",
    tiles = "OSM")

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df['Max Temp'] <= 27) & (city_data_df['Max Temp'] >= 21) &
                                (city_data_df['Wind Speed'] <= 4.5) & (city_data_df['Cloudiness'] < 10)]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,58,terra rica,-22.7094,-52.6169,24.80,66,0,3.52,BR,1713735545
64,64,iquique,-20.2208,-70.1431,23.02,77,3,3.23,CL,1713735095
120,120,thabazimbi,-24.5917,27.4116,21.20,33,5,1.30,ZA,1713735579
143,143,arraial do cabo,-22.9661,-42.0278,23.92,73,0,3.60,BR,1713735594
191,191,la'ie,21.6477,-157.9253,26.98,69,6,4.02,US,1713735592
239,239,armacao de buzios,-22.7469,-41.8817,24.03,74,0,4.26,BR,1713735696
250,250,saint-philippe,-21.3585,55.7679,23.39,87,7,1.90,RE,1713735587
268,268,zouerate,22.7187,-12.4521,24.87,20,0,4.27,MR,1713735746
297,297,karratha,-20.7377,116.8463,22.68,38,0,2.86,AU,1713735558
362,362,susanville,40.4163,-120.6530,24.91,26,0,1.79,US,1713735674


### Step 3: Create a new DataFrame called `hotel_df`.

In [40]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
hotel_df.index.names = ['City_ID']
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

C:\Users\dsfac\AppData\Local\Temp\ipykernel_29556\2216790917.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
58,terra rica,BR,-22.7094,-52.6169,66,
64,iquique,CL,-20.2208,-70.1431,77,
120,thabazimbi,ZA,-24.5917,27.4116,33,
143,arraial do cabo,BR,-22.9661,-42.0278,73,
191,la'ie,US,21.6477,-157.9253,69,
239,armacao de buzios,BR,-22.7469,-41.8817,74,
250,saint-philippe,RE,-21.3585,55.7679,87,
268,zouerate,MR,22.7187,-12.4521,20,
297,karratha,AU,-20.7377,116.8463,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
category = "accommodation.hotel"
filter = ""
bias = ""

params = {
    "categories":category,
    "filter":filter,
    "bias":bias,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
terra rica - nearest hotel: No hotel found
iquique - nearest hotel: Hostal Vivar
thabazimbi - nearest hotel: Maroela Guest Lodge
arraial do cabo - nearest hotel: No hotel found
la'ie - nearest hotel: No hotel found
armacao de buzios - nearest hotel: Vila da Santa
saint-philippe - nearest hotel: Le Baril
zouerate - nearest hotel: فندق تيرس زمور
karratha - nearest hotel: Karratha International Hotel
susanville - nearest hotel: SureStay Plus Hotel By Best Western Susanville
bandar-e lengeh - nearest hotel: Old owner do not like foreigners hotel
serdar - nearest hotel: Wepa
duba - nearest hotel: No hotel found
moab - nearest hotel: Kokopelli Inn
sao joao da barra - nearest hotel: Pousada Mediterrâneo


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
58,terra rica,BR,-22.7094,-52.6169,66,No hotel found
64,iquique,CL,-20.2208,-70.1431,77,Hostal Vivar
120,thabazimbi,ZA,-24.5917,27.4116,33,Maroela Guest Lodge
143,arraial do cabo,BR,-22.9661,-42.0278,73,No hotel found
191,la'ie,US,21.6477,-157.9253,69,No hotel found
239,armacao de buzios,BR,-22.7469,-41.8817,74,Vila da Santa
250,saint-philippe,RE,-21.3585,55.7679,87,Le Baril
268,zouerate,MR,22.7187,-12.4521,20,فندق تيرس زمور
297,karratha,AU,-20.7377,116.8463,38,Karratha International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [50]:

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True,
    frame_width = 800,
    frame_height = 600,
    hover_cols = ["Hotel Name", "Country"],
    size = 50,
    color = "City",
    tiles = "OSM")

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)